# Tasking

[CSDA](https://www.earthdata.nasa.gov/about/csda) supports tasking commercial satellites.
"Tasking" is the process of submitting a request to the commercial satellite vendor to collect data over a specific area within a specific time window (and potentially other constraints).
Tasking in CSDA is a two step process:

1. Tasking proposal: A user submits a proposal to task a satellite. The review board assesses the proposal and either approves or denies it.
2. Tasking request: If their tasking proposal was approved, a user then can make one or more tasking requests against that proposal. The data are collected by the commercial satellite vendor, delivered to CSDA, and made available for download by the user.

The tasking workflow is _usually_ done via the [CSDA frontend](https://csdap.earthdata.nasa.gov/), but this notebook demonstrates how you might do it programmatically.
Note that tasking is under active development, so we'll be demonstrating its usage against our **staging** environment. 

## Setup

For all of these operations, we'll need a `CsdaClient`.
This client will log in to the [Earthdata](https://urs.earthdata.nasa.gov) using either basic (username and password) or [netrc](https://www.gnu.org/software/inetutils/manual/html_node/The-_002enetrc-file.html) authentication.
In this example, we'll assume you've set up your `.netrc` file per [the documentation](https://nsidc.org/data/user-resources/help-center/creating-netrc-file-earthdata-login).


In [9]:
import datetime
from tempfile import NamedTemporaryFile

import json_schema_for_humans.generate
import tabulate
from geojson_pydantic import Point
from geojson_pydantic.types import Position2D
from httpx import NetRCAuth
from IPython.display import Markdown
from json_schema_for_humans.generation_configuration import GenerationConfiguration
from stapi_pydantic import OrderPayload

from csda_client import STAGING_URL, CsdaClient
from csda_client.models import (
    CreateTaskingProductRequest,
    CreateTaskingProposal,
    OrderParameters,
)

In [10]:
client = CsdaClient.open(NetRCAuth(), url=STAGING_URL)
client.verify()

'Hello gadomski, you have a valid token!'

## Tasking proposal

To create a new proposal, we first need to assemble a few things:

- The `Grant` that we'll be associating with this proposal
- One or more vendor's products

First, let's find what grants we have available.
If you're running this notebook yourself, make sure to change this value to be your own Earthdata username.

In [11]:
rows = []
grants = {}
for grant in client.profile("gadomski").grants:
    grants[grant.grant_number] = grant
    rows.append([grant.grant_number, grant.start_date, grant.end_date])
tabulate.tabulate(
    rows, headers=["Grant number", "Start date", "End date"], tablefmt="html"
)

Grant number,Start date,End date
42,2025-05-29,2026-05-29



Now let's list the vendors and find all of them that have tasking enabled.

In [12]:
rows = []
vendors = {}
for vendor in client.vendors():
    if vendor.has_tasking:
        vendors[vendor.slug] = vendor
        rows.append([vendor.id, vendor.slug, vendor.full_name])
tabulate.tabulate(rows, headers=["Id", "Slug", "Full name"], tablefmt="html")

Id,Slug,Full name
12,ghgsat,GHGSat
17,capellaspace,Capella Space
18,iceye,ICEYE US
34,umbra,Umbra Space
47,airbus,Airbus U.S.


Let's create a proposal to task [Umbra Space](https://umbra.space/).
We'll need to see what products are available.

In [13]:
rows = []
products = {}
for product in client.products(vendor_id=vendors["umbra"].id):
    products[product.slug] = product
    rows.append([product.id, product.slug, product.name, product.long_desc])
tabulate.tabulate(
    rows, headers=["Id", "Slug", "Name", "Long description"], tablefmt="html"
)

Id,Slug,Name,Long description
72,CPHD,Compensated Phase History Data,"CPHD product preserves raw SAR echo data before image formation, useful for advanced processing tasks"
76,CSI-SIDD,Color Single Image SIDD,It provides clarity and context to spot objects like buildings in a forest or cars moving on a field.
77,CSI,Color Single Image TIFF,A specialized Color Single Image TIFF product using a three-color (RGB) sub-aperture processing approach to enhance detail.
74,SIDD,Sensor Independent Detected Data,"SIDD offers ground-plane imagery optimized for viewing, omitting phase data but preserving detailed visual information."
75,GEC,Geocoded Ellipsoid Corrected,"This is a radiometrically corrected SAR image with map projection, allowing for easy viewing in common image applications."
73,SICD,Sensor Independent Complex Data,SICD is a slant-plane image retaining SAR phase and magnitude data


Let's ask for a "Color Single Image TIFF".
Here's how to make a tasking proposal.

In [14]:
create_tasking_proposal = CreateTaskingProposal(
    name="An excellent tasking proposal",
    grant=grants["42"].id,
    products=[
        CreateTaskingProductRequest(product=products["CSI"].id, n_proposed_granules=42)
    ],
    research_description="Very important things",
    tasking_justification="It just won't be collected unless I ask for it",
)
tasking_proposal = client.create_tasking_proposal(create_tasking_proposal, submit=False)  # We'll just submit this as a draft, since usually you'll be creating a proposal via the frontend
tasking_proposal

TaskingProposal(id=795, products=[TaskingProductRequest(product=Product(id=77, slug='CSI', name='Color Single Image TIFF', long_desc='A specialized Color Single Image TIFF product using a three-color (RGB) sub-aperture processing approach to enhance detail.'), n_proposed_granules=42, n_allocated_granules=None)], is_draft=True, grant=Grant(id=534, grant_number='42', start_date=datetime.date(2025, 5, 29), end_date=datetime.date(2026, 5, 29)), user='gadomski', name='An excellent tasking proposal', research_description='Very important things', tasking_justification="It just won't be collected unless I ask for it", final_decision_type='', decision_details=None)

## Tasking requests

Once a tasking proposal is approved by the board, research users can submit tasking requests against that a proposal.
Each vendor has its own parameters that can be provided when creating a request.
Let's see what parameters are available for Umbra Space.

In [15]:
config = GenerationConfiguration(template_name="md" ,show_toc=False)

order_parameters = client.get_tasking_order_parameters("umbra-sar")  # There's currently no way to derive this slug, you just have to know it: https://github.com/NASA-IMPACT/csda-project/issues/1682
with NamedTemporaryFile("w", delete_on_close=False) as f:
    f.write(order_parameters.model_dump_json())
    f.close()
    display(Markdown(json_schema_for_humans.generate.generate_from_schema(f.name, config=config)))

== Generating tmpc_fw1_z5 ==
== Generated tmpc_fw1_z5 in 0:00:00.019612 ==


# OrderParameters

**Title:** OrderParameters

|                           |             |
| ------------------------- | ----------- |
| **Type**                  | `object`    |
| **Required**              | No          |
| **Additional properties** | Not allowed |

| Property                                   | Pattern | Type              | Deprecated | Definition                 | Title/Description |
| ------------------------------------------ | ------- | ----------------- | ---------- | -------------------------- | ----------------- |
| + [proposal_id](#proposal_id )             | No      | integer           | No         | -                          | Proposal Id       |
| + [product_ids](#product_ids )             | No      | array of integer  | No         | -                          | Product Ids       |
| + [task_name](#task_name )                 | No      | string            | No         | -                          | Task Name         |
| - [imaging_mode](#imaging_mode )           | No      | enum (of string)  | No         | In #/$defs/ImagingMode     | Imaging Mode      |
| - [grazing_angle_min](#grazing_angle_min ) | No      | integer           | No         | -                          | Grazing Angle Min |
| - [grazing_angle_max](#grazing_angle_max ) | No      | integer           | No         | -                          | Grazing Angle Max |
| - [multilook_factor](#multilook_factor )   | No      | enum (of integer) | No         | In #/$defs/MultilookFactor | Multilook Factor  |

## <a name="proposal_id"></a>1. Property `OrderParameters > proposal_id`

**Title:** Proposal Id

|              |           |
| ------------ | --------- |
| **Type**     | `integer` |
| **Required** | Yes       |

## <a name="product_ids"></a>2. Property `OrderParameters > product_ids`

**Title:** Product Ids

|              |                    |
| ------------ | ------------------ |
| **Type**     | `array of integer` |
| **Required** | Yes                |

|                      | Array restrictions |
| -------------------- | ------------------ |
| **Min items**        | 1                  |
| **Max items**        | N/A                |
| **Items unicity**    | False              |
| **Additional items** | False              |
| **Tuple validation** | See below          |

| Each item of this array must be         | Description |
| --------------------------------------- | ----------- |
| [product_ids items](#product_ids_items) | -           |

### <a name="product_ids_items"></a>2.1. OrderParameters > product_ids > product_ids items

|              |           |
| ------------ | --------- |
| **Type**     | `integer` |
| **Required** | No        |

## <a name="task_name"></a>3. Property `OrderParameters > task_name`

**Title:** Task Name

|              |          |
| ------------ | -------- |
| **Type**     | `string` |
| **Required** | Yes      |

## <a name="imaging_mode"></a>4. Property `OrderParameters > imaging_mode`

**Title:** Imaging Mode

|                |                     |
| -------------- | ------------------- |
| **Type**       | `enum (of string)`  |
| **Required**   | No                  |
| **Default**    | `"SPOTLIGHT"`       |
| **Defined in** | #/$defs/ImagingMode |

Must be one of:
* "SPOTLIGHT"
* "SCAN"

## <a name="grazing_angle_min"></a>5. Property `OrderParameters > grazing_angle_min`

**Title:** Grazing Angle Min

|              |           |
| ------------ | --------- |
| **Type**     | `integer` |
| **Required** | No        |
| **Default**  | `45`      |

## <a name="grazing_angle_max"></a>6. Property `OrderParameters > grazing_angle_max`

**Title:** Grazing Angle Max

|              |           |
| ------------ | --------- |
| **Type**     | `integer` |
| **Required** | No        |
| **Default**  | `65`      |

## <a name="multilook_factor"></a>7. Property `OrderParameters > multilook_factor`

**Title:** Multilook Factor

|                |                         |
| -------------- | ----------------------- |
| **Type**       | `enum (of integer)`     |
| **Required**   | No                      |
| **Default**    | `1`                     |
| **Defined in** | #/$defs/MultilookFactor |

Must be one of:
* 1
* 2
* 4
* 8

----------------------------------------------------------------------------------------------------------------------------
Generated using [json-schema-for-humans](https://github.com/coveooss/json-schema-for-humans) on 2025-08-28 at 11:09:49 -0600


The required fields are:

- `proposal_id`, which we have
- `product_ids`, which we have from via our proposal
- `task_name`, which we can make up

Easy enough!
Let's create a tasking request, which is also a [STAPI order](https://github.com/stapi-spec/stapi-spec/tree/main/order#order-object).
We'll create the request for a time period starting tomorrow and ending in a week, over [Longmont, Colorado, USA](https://en.wikipedia.org/wiki/Longmont,_Colorado).

In [16]:
order_parameters = OrderParameters.model_validate({
    "proposal_id": tasking_proposal.id,
    "product_ids": [products["CSI"].id],
    "task_name": "My tasking proposal"
})
order_payload = OrderPayload(
    datetime=(
        datetime.datetime.now(tz=datetime.timezone.utc) + datetime.timedelta(days=1),
        datetime.datetime.now(tz=datetime.timezone.utc) + datetime.timedelta(days=7),
    ),
    geometry=Point(type="Point", coordinates=Position2D(-105.1019, 40.1672)),
    order_parameters=order_parameters,
)
order = client.create_tasking_request("umbra-sar", order_payload)
order

Order(bbox=None, id='aefb49ae-0277-405f-acb3-7be023ba92aa', type='Feature', stapi_type='Order', stapi_version='0.1.0', geometry=Point(bbox=None, type='Point', coordinates=Position2D(longitude=-105.1019, latitude=40.1672)), properties=OrderProperties(product_id='umbra-sar', created=datetime.datetime(2025, 8, 28, 17, 9, 49, 136199, tzinfo=TzInfo(UTC)), status=OrderStatus(timestamp=datetime.datetime(2025, 8, 28, 17, 9, 49, 136199, tzinfo=TzInfo(UTC)), status_code=<OrderStatusCode.received: 'received'>, reason_code=None, reason_text=None, links=[]), search_parameters=OrderSearchParameters(datetime=(datetime.datetime(2025, 8, 29, 17, 9, 49, 36402, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 9, 4, 17, 9, 49, 36406, tzinfo=datetime.timezone.utc)), geometry=Point(bbox=None, type='Point', coordinates=Position2D(longitude=-105.1019, latitude=40.1672)), filter=None), opportunity_properties={}, order_parameters={'task_name': 'My tasking proposal', 'imaging_mode': 'SPOTLIGHT', 'grazing_a

After a tasking request is submitted  it will be vetted, first by an automatic system and then by CSDA operations.
If it's valid, it will be sent on to the vendor via their specific tasking API.
When the tasking request is fulfilled, CSDA will notify the research user.

## Accessing your tasked data

TODO